In [4]:
import pandas as pd

In [3]:
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf
stock_symbol = "RELIANCE.NS"  #NSE stocks usually end with .NS
#download stock data
stk_data = yf.download(stock_symbol,start="2023-01-01",end="2023-07-01")

[*********************100%***********************]  1 of 1 completed


In [5]:
stk_data=stk_data[["Open","High","Low","Close"]]

In [7]:
#column="Close"

In [6]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (122, 4)


In [7]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [8]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

98
X_train length: (98, 4)
X_test length: (24, 4)
y_train length: (98, 4)
y_test length: (24, 4)


In [9]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [16]:
def combination(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]  #selecting only the columns in listt
    
    #train test split
    #Last 28 observations are used as test data. The rest is training data
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]

    #model selection loop
    #Fit the VAR model with lags from 1 to 10. Print AIC and BIC for each to evaluate which lag performs best
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()

    #Automatically select the best lag using Akaike Information Criterion (AIC)
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]

    #Fit the final model
    #Fit VAR using the best lag. Get the last order rows of training data for forecasting
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]

    #forecasting
    #Forecast 28 future steps.Convert the result into a DataFrame
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)

    #Save the forecast to a CSV file for reference
    preds.to_csv("varforecasted_{}.csv".format(test_obs))

    #evaluation: ompute RMSE and MAPE between predicted and actual values
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)

    #Append performance metrics into a shared performance dictionary (assumed to be global).
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [11]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [18]:
perf,result,pred=combination(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -27.476716924903812
BIC:  -26.932071872612788

Order = 2
AIC:  -27.416568270570608
BIC:  -26.429781436073156

Order = 3
AIC:  -27.354582772370158
BIC:  -25.919805911503385

Order = 4
AIC:  -27.130750543031553
BIC:  -25.24200545878202

Order = 5
AIC:  -26.93648253233639
BIC:  -24.587657194611673

Order = 6
AIC:  -26.92323335557875
BIC:  -24.108077884580787

Order = 7
AIC:  -27.02282299745829
BIC:  -23.734945505918848

Order = 8
AIC:  -26.93410352963207
BIC:  -23.166965819103428

Order = 9
AIC:  -26.753041624066675
BIC:  -22.49995473041295

Order = 10
AIC:  -26.91609324793979
BIC:  -22.170212831150465



In [19]:
data1

,Open,High,Low,Close
0,0.856777,0.930052,0.968437,0.947163
1,0.895268,0.914508,0.967385,0.899507
2,0.874681,0.883550,0.878485,0.802175
3,0.789003,0.819690,0.852183,0.790798
4,0.797059,0.849611,0.889795,0.848565
...,...,...,...,...
117,0.802813,0.811917,0.873224,0.792568
118,0.746803,0.765285,0.811941,0.744028
119,0.720077,0.750000,0.806550,0.746303
120,0.762149,0.823446,0.836007,0.829858


In [20]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.327094,1,28
